In [2]:
from langchain_community.document_loaders import TextLoader     #converts text into langchain compatible format
from langchain_text_splitters import CharacterTextSplitter      #splits whole document into meaningful chunks, in this case descriptions
from langchain_openai import OpenAIEmbeddings                   #handles the creation of vectorised embedings of description chunks
from langchain_chroma import Chroma                             #stores embedings in a vector database

In [ ]:
#SETUP dotenv TO IMPORT API KEYS HERE

In [3]:
import pandas as pd
#importing the nice and clean dataset
books = pd.read_csv('books_cleaned.csv')

In [5]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [8]:
#langchain cannot access information in a pandas dataframe
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep="\n",
                                   index=False,
                                   header=False)
#this will save the tagged descriptions into a langchain compatible txt file